In [2]:
import subprocess as sp

# Print the return code (status=0 mean ping success)
status = sp.call(['tcping', '-c', '1', '-t', '1', 'github.com'], stdout=sp.DEVNULL, stderr=sp.DEVNULL)
print(status)

# OR print the full message
status = sp.run(['tcping', '-c', '1', '-t', '1', 'github.com'], stdout=sp.DEVNULL, stderr=sp.DEVNULL)
print(status)

0
CompletedProcess(args=['tcping', '-c', '1', '-t', '1', 'github.com'], returncode=0)


In [5]:
from tcping import Ping

def ping_check():
    ping = Ping('18.162.184.7', 80, 1)
    ping.ping(10)

    ret = ping.result.rows
    for r in ret:
        print(r)

    ret = ping.result.raw
    print(ret)

    ret = ping.result.table
    print(ret)

ping_check()

Connected to 18.162.184.7[:80]: seq=1 time=101.48 ms
Connected to 18.162.184.7[:80]: seq=2 time=123.42 ms
Connected to 18.162.184.7[:80]: seq=3 time=102.33 ms
Connected to 18.162.184.7[:80]: seq=4 time=100.05 ms
Connected to 18.162.184.7[:80]: seq=5 time=102.30 ms
Connected to 18.162.184.7[:80]: seq=6 time=107.81 ms
Connected to 18.162.184.7[:80]: seq=7 time=104.76 ms
Connected to 18.162.184.7[:80]: seq=8 time=116.21 ms
Connected to 18.162.184.7[:80]: seq=9 time=118.27 ms
Connected to 18.162.184.7[:80]: seq=10 time=121.65 ms
Statistics(host='18.162.184.7', port=80, successed=10, failed=0, success_rate='100.00%', minimum='100.05ms', maximum='123.42ms', average='109.83ms')

--- 18.162.184.7[:80] tcping statistics ---
10 connections, 10 successed, 0 failed, 100.00% success rate
minimum = 100.05ms, maximum = 123.42ms, average = 109.83ms

+--------------+------+-----------+--------+--------------+----------+----------+----------+
|     Host     | Port | Successed | Failed | Success Rate | M

In [ ]:
import requests

# header = {"Host": 'www.okx.com'}
# url = 'https://13.35.125.81/api/v5/public/time'
# response = requests.get(url, headers=header, verify=False)
# print(response.json())


class HostHeaderSSLAdapter(requests.adapters.HTTPAdapter):
    def resolve(self, hostname):
        # a dummy DNS resolver
        import random
        ips = [
            # '104.16.89.20',  # CloudFlare
            # '151.101.2.109',  # Fastly
            '16.163.225.163',
        ]
        resolutions = {
            'www.okx.com': random.choice(ips),
        }
        return resolutions.get(hostname)

    def send(self, request, **kwargs):
        from urllib.parse import urlparse

        connection_pool_kwargs = self.poolmanager.connection_pool_kw

        result = urlparse(request.url)
        resolved_ip = self.resolve(result.hostname)

        if result.scheme == 'https' and resolved_ip:
            request.url = request.url.replace(
                'https://' + result.hostname,
                'https://' + resolved_ip,
            )
            connection_pool_kwargs['server_hostname'] = result.hostname  # SNI
            connection_pool_kwargs['assert_hostname'] = result.hostname

            # overwrite the host header
            request.headers['Host'] = result.hostname
        else:
            # theses headers from a previous request may have been left
            connection_pool_kwargs.pop('server_hostname', None)
            connection_pool_kwargs.pop('assert_hostname', None)

        return super(HostHeaderSSLAdapter, self).send(request, **kwargs)



# url = 'https://cdn.jsdelivr.net/npm/bootstrap/LICENSE'
url = 'https://www.okx.com/api/v5/public/time'

session = requests.Session()
session.mount('https://', HostHeaderSSLAdapter())

r = session.get(url)
print(r.headers)




In [12]:
import requests
from forcediphttpsadapter.adapters import ForcedIPHTTPSAdapter
requests.urllib3.disable_warnings() 

session = requests.Session()
session.mount("https://www.okx.com", ForcedIPHTTPSAdapter(dest_ip='13.35.125.81'))
response = session.get(
    'https://www.okx.com/api/v5/public/time', verify=False)

print( response.json() )

{'code': '0', 'data': [{'ts': '1663053562450'}], 'msg': ''}


In [ ]:
import requests
from requests_toolbelt.adapters.host_header_ssl import HostHeaderSSLAdapter

s = requests.session()
s.mount('https://', HostHeaderSSLAdapter())
res = s.get("https://13.35.125.81/api/v5/public/time", headers={"Host": "www.okx.com"})
print( res.json() )

